<a href="https://colab.research.google.com/github/ZaimaSartajTaheri/Blood_Donate/blob/master/Taheri_Thesis_7_Chatgpt_V1(Aug_Specific_Layer_VGG16_%26_CNN_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D,GlobalMaxPooling2D, concatenate, Dropout,Conv2D, MaxPooling2D, Flatten

In [4]:
train_dataset = pd.read_csv('/content/drive/MyDrive/TaheriThesis/Dataset/trainA.csv')

In [5]:
def load_image(path):
    img = load_img(path, target_size=(224,224))
    img = img_to_array(img)
    img = img / 255.0
    return img

In [13]:
import nltk
nltk.download('wordnet')
import imgaug.augmenters as iaa
from nltk.corpus import wordnet


def preprocess_images(data):
    images = []

    augmenter = iaa.Sequential([
        iaa.Fliplr(0.5),  # flip horizontally with 50% probability
        iaa.Affine(rotate=(-15, 15)),  # rotate the image by -15 to 15 degrees
        iaa.GaussianBlur(sigma=(0, 3.0)),  # apply Gaussian blur with sigma between 0 and 3.0
        iaa.Multiply((0.8, 1.2))  # change brightness of the image randomly
    ])
    for index, row in data.iterrows():
        # Image preprocessing and augmentation
        path = os.path.join('/content/drive/MyDrive/TaheriThesis/Dataset/Images/', row['image_name'])
        img = load_image(path)
        img_aug = augmenter.augment_image(img)  # apply image augmentation to the image
        images.append(img_aug)
        
    
    
    images = np.array(images)
  
    return images


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [14]:


def preprocess_texts(data):
    
    texts = []
    
    for index, row in data.iterrows():
       
        
        # Text preprocessing and augmentation
        text = row['Captions']
        words = text.split()
        if len(words) > 1:
            # Synonym replacement
            new_words = []
            for word in words:
                synsets = wordnet.synsets(word)
                if synsets:
                    synset = synsets[0]
                    synonyms = synset.lemma_names()
                    if len(synonyms) > 1:
                        new_word = synonyms[1]
                        new_words.append(new_word)
                    else:
                        new_words.append(word)
                else:
                    new_words.append(word)
            new_text = " ".join(new_words)
            
            # Random insertion/deletion
            if np.random.rand() < 0.5:
                # Insert a random word
                new_words.insert(np.random.randint(len(new_words)), "RANDOM_WORD")
            else:
                # Delete a random word
                del new_words[np.random.randint(len(new_words))]
            new_text = " ".join(new_words)
            
            # Character level augmentation
            new_text = new_text.lower()
            new_text = new_text.replace(".", "")
            new_text = new_text.replace(",", "")
            new_text = new_text.replace("?", "")
            new_text = new_text.replace("!", "")
            new_text = new_text.replace("-", "")
            new_text = new_text.replace("'", "")
            new_text = new_text.replace('"', '')
            new_text = list(new_text)
            for i in range(len(new_text)):
                if np.random.rand() < 0.1:
                    new_text[i] = new_text[i].upper()
            new_text = "".join(new_text)
            texts.append(new_text)
        else:
            texts.append(text)
    
   
    texts = np.array(texts)
    return texts


In [15]:
# def preprocess_images(data):
#     images = []
#     for path in data['image_name']:
#         path = os.path.join('/content/drive/MyDrive/TaheriThesis/Dataset/Images/', path)
#         img = load_image(path)
#         images.append(img)
#     images = np.array(images)
#     return images

In [16]:
from sklearn.utils import shuffle
train_data = shuffle(train_dataset, random_state = 10)
labels = train_data['Label']
labels

3297    6
176     4
2994    6
774     0
1578    1
       ..
2009    4
1180    0
3441    6
1344    1
1289    1
Name: Label, Length: 4143, dtype: int64

In [17]:
# import the necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.layers import Input, Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, concatenate
from tensorflow.keras.layers import Input, Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, concatenate, Embedding
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical

# define the hyperparameters
max_sequence_length = 50 # max sequence length for text
embedding_dim = 100 # embedding dimension for text
max_words = 20000 # max number of words in the tokenizer
num_classes = 7 # number of emotion classes
image_size = (224, 224) # size of input image for the VGG16 model

# load the VGG16 model with pre-trained weights and without the last layer
image_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# unfreeze the last convolutional block of the VGG16 model
for layer in image_model.layers[:-5]:
    layer.trainable = False

# define the text input
text_input = Input(shape=(max_sequence_length,), dtype='int32')

# create the text embedding layer
text_embedding = Embedding(max_words, embedding_dim, input_length=max_sequence_length)(text_input)

# # apply a 1D convolutional layer and max pooling layer to the text embedding
# text_conv = Conv1D(128, 5, activation='relu')(text_embedding)
# text_pool = MaxPooling1D(5)(text_conv)
# text_drop = Dropout(0.2)(text_pool)
# text_lstm = LSTM(100)(text_drop)

# # get the output of the image model
# image_output = image_model.output
# image_flat = Flatten()(image_output)
# image_dense = Dense(128, activation='relu')(image_flat)
# image_drop = Dropout(0.2)(image_dense)

# # concatenate the text and image features
# merged = concatenate([image_drop, text_lstm])

# # apply a final fully connected layer to get the output probabilities
# output = Dense(num_classes, activation='softmax')(merged)

# # create the final model
# model = Model(inputs=[image_model.input, text_input], outputs=output)

# # compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # load the data
# train_data = shuffle(train_dataset, random_state = 10)
# images = preprocess_images(train_data)
# texts = train_data['Captions']
# labels = train_data['Label']
# labels = to_categorical(labels)

# # split the data into training and testing sets
# train_images, test_images, train_texts, test_texts, train_labels, test_labels = train_test_split(images, texts, labels, test_size=0.2, random_state=42)

# # tokenize the text data
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(train_texts)
# train_sequences = tokenizer.texts_to_sequences(train_texts)
# test_sequences = tokenizer.texts_to_sequences(test_texts)
# train_text_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
# test_text_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# # train the model
# model.fit([train_images, train_text_data], train_labels, batch_size=64, epochs=10, validation_data=([test_images, test_text_data], test_labels))

# score, acc = model.evaluate([test_images, test_text_data], test_labels, batch_size=64)
# print('Test score:', score)
# print('Test accuracy:', acc)

# # from tensorflow.keras.models import load_model
# # loaded_model = load_model('emotion_detection_model.h5')


In [29]:
# apply a 1D convolutional layer and max pooling layer to the text embedding
text_conv = Conv1D(128, 5, activation='relu')(text_embedding)
text_pool = MaxPooling1D(5)(text_conv)
text_drop = Dropout(0.2)(text_pool)
text_lstm = LSTM(100)(text_drop)

# get the output of the image model
image_output = image_model.output
image_flat = Flatten()(image_output)
image_dense = Dense(128, activation='relu')(image_flat)
image_drop = Dropout(0.2)(image_dense)

# concatenate the text and image features
merged = concatenate([image_drop, text_lstm])

# apply a final fully connected layer to get the output probabilities
output = Dense(num_classes, activation='softmax')(merged)

# create the final model
model = Model(inputs=[image_model.input, text_input], outputs=output)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
train_data = shuffle(train_dataset, random_state = 10)
images = preprocess_images(train_data)

In [21]:
# load the data

texts =  preprocess_texts(train_data)
labels = train_data['Label']
labels = to_categorical(labels)

In [22]:
# split the data into training and testing sets
train_images, test_images, train_texts, test_texts, train_labels, test_labels = train_test_split(images, texts, labels, test_size=0.1, random_state=42)

# tokenize the text data
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
train_text_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_text_data = pad_sequences(test_sequences, maxlen=max_sequence_length)


In [ ]:
# train the model
# model.fit([train_images, train_text_data], train_labels, batch_size=32, epochs=10, validation_data=([test_images, test_text_data], test_labels))
model.fit([train_images, train_text_data], train_labels, batch_size=32, epochs=8, validation_split =0.1)


In [24]:
y_test= np.array(test_labels)

In [25]:
score, acc = model.evaluate([test_images, test_text_data], test_labels, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

# from tensorflow.keras.models import load_model
# loaded_model = load_model('emotion_detection_model.h5')

13/13 [==============================] - 6s 483ms/step - loss: 3.2697 - accuracy: 0.3952
Test score: 3.2696714401245117
Test accuracy: 0.39518073201179504


In [26]:
# type(y_pred_test)

In [27]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix
def print_metrices(true,pred):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,target_names=['happy','angry','disgust','fear','sad','surprise','other']))
    print("Accuracy : ",accuracy_score(true,pred))
    print("Precison : ",precision_score(true,pred, average = 'weighted'))
    print("Recall : ",recall_score(true,pred,  average = 'weighted'))
    print("F1 : ",f1_score(true,pred,  average = 'weighted'))

In [28]:
pred_test = model.predict([test_images, test_text_data])
y_pred_test = np.argmax(pred_test, axis=1)
y_test_single = np.argmax(y_test, axis=1)

print_metrices(y_test_single, y_pred_test)

13/13 [==============================] - 2s 116ms/step
[[36  2  1  9 10  5 14]
 [ 3 19  1  5 11  5 11]
 [ 3  4 14  3  4  3  6]
 [ 3  2  1 32  3  4  4]
 [10 11  3  4 24  7 12]
 [15  2  3  5  4 13  9]
 [13 12  1  9  7  7 26]]
              precision    recall  f1-score   support

       happy       0.43      0.47      0.45        77
       angry       0.37      0.35      0.36        55
     disgust       0.58      0.38      0.46        37
        fear       0.48      0.65      0.55        49
         sad       0.38      0.34      0.36        71
    surprise       0.30      0.25      0.27        51
       other       0.32      0.35      0.33        75

    accuracy                           0.40       415
   macro avg       0.41      0.40      0.40       415
weighted avg       0.40      0.40      0.39       415

Accuracy :  0.39518072289156625
Precison :  0.3960875934608163
Recall :  0.39518072289156625
F1 :  0.3914031192417335
